In [1]:
!pip install torch torchaudio phonemizer
!pip install soundfile
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 28.4 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=df78f45934047530b46fc1585cc665f928360796582346787c3a37131c0f7ca1
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [6]:
from transformers import VitsTokenizer, VitsModel, Trainer, TrainingArguments, set_seed
import torch
import torchaudio
import json
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset


In [3]:
with open('/content/562949953671638.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

segments = data['verbatim']
texts = [segment['text'] for segment in segments]
start_times = [segment['start'] for segment in segments]
end_times = [segment['end'] for segment in segments]
durations = [end - start for start, end in zip(start_times, end_times)]

In [7]:
from transformers import VitsTokenizer, VitsModel

# Initialize tokenizer and model
tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")
model = VitsModel.from_pretrained("facebook/mms-tts-eng")
from transformers import DataCollatorWithPadding

class MarathiTTSDataset(Dataset):
    def __init__(self, texts, durations, tokenizer): # Add tokenizer as an argument
        self.texts = texts
        self.durations = durations
        self.tokenizer = tokenizer # Store the tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        duration = self.durations[idx]

        # Tokenize the text using the tokenizer
        inputs = self.tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")

        # Return a dictionary containing the tokenized input IDs and duration
        return {
            'input_ids': inputs.input_ids.squeeze(0),  # Remove batch dimension
            'attention_mask': inputs.attention_mask.squeeze(0), # Remove batch dimension
            'duration': duration
        }

# Pass the tokenizer when creating the dataset instance
dataset = MarathiTTSDataset(texts, durations, tokenizer) # Pass tokenizer here
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=data_collator)

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [8]:
from torch.optim import Adam
import torch

optimizer = Adam(model.parameters(), lr=1e-4)
epochs = 5
loss_fn = torch.nn.MSELoss()

for epoch in range(epochs):
    for batch in dataloader:
        inputs = {
            'input_ids': batch['input_ids'],
            'attention_mask': batch['attention_mask'],
        }

        # Generate waveforms using the model
        outputs = model(**inputs)
        target_audio = torch.randn_like(outputs.waveform)
        loss = loss_fn(outputs.waveform, target_audio)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item()}")

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch 1/5 - Loss: 1.00327467918396
Epoch 2/5 - Loss: 0.9880967736244202
Epoch 3/5 - Loss: 1.0034801959991455
Epoch 4/5 - Loss: 1.0064421892166138
Epoch 5/5 - Loss: 1.0022525787353516
